In [1]:
import urllib,urllib.request,sys,json
import pandas as pd
import os
import socket
import hashlib
import time
# timeout in seconds
from functools import reduce
import urllib,urllib.request,sys,json
import pandas as pd
import os
import socket
import hashlib
import time
# timeout in seconds
from functools import reduce
timeout = 5
socket.setdefaulttimeout(timeout)

def md5_entropy(str_x):
    hl = hashlib.md5()
    hl.update(str_x.encode(encoding='utf-8'))
    return hl.hexdigest()


def weather_city(cityid,search):
    #获取天气数据，cityid为城市ID,str（如“285332”，详见cityid.xml），search为获取类型,str（如“天气实况”）。
    appcode = 'b9347b22f24442e29d620b579d701e23'
    host = 'http://aliv18.data.moji.com' # 15days
    # host = 'http://coapi.moji.com'   # 40days
    path = {"天气实况":"/whapi/json/alicityweather/condition", #查询项目对应的地址
    "天气预报24小时":"/whapi/json/alicityweather/forecast24hours",
    "天气预报15天":"/whapi/json/alicityweather/forecast15days",
    "空气质量指数":"/whapi/json/alicityweather/aqi",
    "限行数据":"/whapi/json/alicityweather/limit",
    "AQI预报5天":"/whapi/json/alicityweather/aqiforecast5days",
    "天气预警":"/whapi/json/alicityweather/alert",
    "生活指数":"/whapi/json/alicityweather/index",
    "天气预报40天":"/whapi/json/weather"
    }
    token = {"天气实况":"50b53ff8dd7d9fa320d3d3ca32cf8ed1", #token为默认项目
    "天气预报24小时":"008d2ad9197090c5dddc76f583616606",
    "天气预报15天":"f9f212e1996e79e0e602b08ea297ffb0",
    "空气质量指数":"8b36edf8e3444047812be3a59d27bab9",
    "限行数据":"27200005b3475f8b0e26428f9bfb13e9",
    "AQI预报5天":"0418c1f4e5e66405d33556418189d2d0",
    "天气预警":"7ebe966ee2e04bbd8cdbc0b84f7f3bc7",
    "生活指数":"5944a84ec4a071359cc4f6928b797f91",
     "天气预报40天": "5478b32f98b9622139c4639f4e279d8a"
    }
    url = host + path[search]  # 15 days
    headers = {'Authorization':'APPCODE ' + appcode,'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8'}
    bodys = {"cityId":cityid,"tioken":token[search]} #  15 days
    time_id = str(int(round(time.time() * 1000)))
    # time_id = str(0)
    md5_key = md5_entropy('61c15808d762733816d166514d45b868' + str(time_id) + str(cityid))
    # bodys = {"cityId":cityid,"token":token[search],'timestamp':time_id,'key':md5_key}  # 40 days
    post_data = urllib.parse.urlencode(bodys).encode('UTF-8')

    url = host + path[search]+'?'+urllib.parse.urlencode(bodys)
    print(str(url))
    try:
        # request = urllib.request.Request(url, post_data,headers) # 40days
        request = urllib.request.Request(url, headers) #15days
        response = urllib.request.urlopen(request)
        content = response.read()
        content = content.decode("UTF-8")
        #分割数据，给city赋值的同时返回data。
        dict_moji = json.loads(content)
    except:
        request = urllib.request.Request(url, post_data,headers)
        response = urllib.request.urlopen(request)
        content = response.read()
        content = content.decode("UTF-8")
        #分割数据，给city赋值的同时返回data。
        dict_moji = json.loads(content)
    return dict_moji['data']
    # return url

def write_json(data,path):
    with open(path,'w+') as f:
        f.write(str(data))
        print(path)
        print('writeing')

def read_json(path):
    with open(path,'r',encoding='gbk') as f:
        tem = f.read()
        print(path)
    return tem


def parse_json(path):
    try:
        json_dict = json.loads(str(read_json(path)).replace("'", "\""))
        cityid = json_dict['city']['cityId']
        name  = json_dict['city']['name']
        pname = json_dict['city']['pname']
        pd_tem = pd.DataFrame(json_dict['forecast']) # 15days
            # pd_tem = pd.DataFrame(json_dict['forecastForty']) # 40days
        pd_tem['cityid'] = cityid
        pd_tem['name'] = name
        pd_tem['pname'] = pname
    except:
        pd_tem = pd.DataFrame(columns=['cityid','name','pname'])
    return pd_tem




def get_40days_da(URL):
    request = urllib.request.Request(URL)
    response = urllib.request.urlopen(request)
    content = response.read()
    content = content.decode("UTF-8")
    # 分割数据，给city赋值的同时返回data。
    dict_moji = json.loads(content)
    return dict_moji['data']

In [2]:
def write_json(data,path):
    with open(path,'w+') as f:
        f.write(str(data))
        print(path)
        print('writeing')

def read_json(path):
    with open(path,'r',encoding='gbk') as f:
        tem = f.read()
        print(path)
    return tem


def parse_json(path):
    try:
        json_dict = json.loads(str(read_json(path)).replace("'", "\""))
        cityid = json_dict['city']['cityId']
        name  = json_dict['city']['name']
        pname = json_dict['city']['pname']
        pd_tem = pd.DataFrame(json_dict['forecast']) # 15days
            # pd_tem = pd.DataFrame(json_dict['forecastForty']) # 40days
        pd_tem['cityid'] = cityid
        pd_tem['name'] = name
        pd_tem['pname'] = pname
    except:
        pd_tem = pd.DataFrame(columns=['cityid','name','pname'])
    return pd_tem

def get_40days_da(URL):
    request = urllib.request.Request(URL)
    response = urllib.request.urlopen(request)
    content = response.read()
    content = content.decode("UTF-8")
    # 分割数据，给city赋值的同时返回data。
    dict_moji = json.loads(content)
    return dict_moji['data']

In [3]:
# 第一步
# 使用说明：运行first step 会根据ori文件夹中的txt文本
# 针对那些filter不为空的地区id数据进行下载 下载在wea15-file 文件夹中
# 第二步
# 运行step2，会汇总在wea15-file 中的数据汇总为一张表在re文件夹中

In [4]:
# first step download file
# data_sheet = pd.read_csv('.\\ori\\moji_used_filter.txt',sep='\t')
# city_code_list=[]
# city_code_list = data_sheet[~data_sheet['Filter'].isnull()]['Fid'].tolist()
# city_code_list_app= [1045,537,1020,2440,2299]
# city_code_list.extend(city_code_list_app)
# save_paths = '.\\wea15-file\\'
# [write_json(weather_city(_,'天气预报15天'),save_paths+ str(_)+'.json')for index,_ in enumerate(city_code_list) if index>=0 ]
# print (len(city_code_list))
# print ('allending')
# data_sheet

In [10]:
data_sheet = pd.read_table('.\\re\\moji_used_filter.txt')
city_code_list=[]
city_code_list = data_sheet['cityid'].tolist()
# city_code_list_app= [1045,537,1020,2440,2299]
# city_code_list.extend(city_code_list_app)
save_paths = '.\\wea15-file\\'
[write_json(weather_city(_,'天气预报15天'),save_paths+ str(_)+'.json')for index,_ in enumerate(city_code_list) if index >=265 ]
print (len(city_code_list))
print ('allending')
# len(city_code_list)

http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=359&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\359.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=379&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\379.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=388&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\388.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=39&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\39.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=394&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\394.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=407&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\407.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=416&tioken=f9f212e19

.\wea15-file\825.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=830&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\830.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=837&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\837.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=841&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\841.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=848&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\848.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=861&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\861.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast15days?cityId=873&tioken=f9f212e1996e79e0e602b08ea297ffb0
.\wea15-file\873.json
writeing
http://aliv18.data.moji.com/whapi/json/alicityweather/forecast1

In [11]:
# second step
save_paths = '.\\wea15-file\\'
all_paths = [os.path.join(save_paths,_) for _ in os.listdir(save_paths)]
all_da = reduce(lambda x,y:pd.concat([x,y],axis=0,ignore_index=True),map(parse_json,all_paths))
all_da.to_csv('.\\re\\all_191008.csv',encoding='gbk')
print('allending')

.\wea15-file\1000.json
.\wea15-file\1002.json
.\wea15-file\1004.json
.\wea15-file\1008.json
.\wea15-file\1014.json
.\wea15-file\1020.json
.\wea15-file\1022.json
.\wea15-file\1024.json
.\wea15-file\104.json
.\wea15-file\1045.json
.\wea15-file\1061.json
.\wea15-file\1068.json
.\wea15-file\1071.json
.\wea15-file\1084.json
.\wea15-file\1093.json
.\wea15-file\1101.json
.\wea15-file\1108.json
.\wea15-file\1119.json
.\wea15-file\113.json
.\wea15-file\1130.json
.\wea15-file\1141.json
.\wea15-file\1152.json
.\wea15-file\1161.json
.\wea15-file\1166.json
.\wea15-file\1169.json
.\wea15-file\1177.json
.\wea15-file\1185.json
.\wea15-file\1194.json
.\wea15-file\1208.json
.\wea15-file\1220.json
.\wea15-file\1233.json
.\wea15-file\124.json
.\wea15-file\1241.json
.\wea15-file\1246.json
.\wea15-file\1253.json
.\wea15-file\1262.json
.\wea15-file\1269.json
.\wea15-file\1275.json
.\wea15-file\1286.json
.\wea15-file\129.json
.\wea15-file\1295.json
.\wea15-file\1306.json
.\wea15-file\1318.json
.\wea15-file\13